<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **テキストデータの扱い方：基本中の基本編**

* テキストデータは、長い長い文字列。
* 長い長い文字列のままでは、普通は分析できない。
* 今回は、自然言語処理における基本的な前処理について学ぶ。
* 今回は、英語データのみを扱う。
 * 日本語データは、次回。


## str型のメソッドによる前処理

* 例えば、大文字と小文字の間の変換などが実行できる。

 * 問：元のテキストにあった大文字と小文字の区別を無くしてしまうことのメリットとデメリットは何か？

In [ ]:
text = 'The quick brown fox jumped over The Big Dog'
text

In [ ]:
text.lower()

In [ ]:
text.upper()

* 各トークンの一文字目を大文字にする。


In [ ]:
text.title()

## NLTK

* NLTKは、Pythonの有名な自然言語処理ライブラリ。2001年スタートらしい。

* https://www.nltk.org/
 * WordNetについては『IT Text 自然言語処理の基礎』3.2.2(a)を参照。

In [ ]:
import nltk

### Tokenization

* 文に分ける、単語に分ける、など、長い文字列としての言語データをより小さな単位へと分割することを、一般にtokenizationと言う。

* Pythonの文字列は、複数行にわたっていても、丸括弧でくくれば一つの長い文字列になる。
 * ただし、最後の行を除いて、末尾に空白を入れておくのを忘れないように。

In [ ]:
sample_text = ("US unveils world's most powerful supercomputer, beats China. " 
               "The US has unveiled the world's most powerful supercomputer called 'Summit', " 
               "beating the previous record-holder China's Sunway TaihuLight. With a peak performance "
               "of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, "
               "which is capable of 93,000 trillion calculations per second. Summit has 4,608 servers, "
               "which reportedly take up the size of two tennis courts.")
sample_text

* 文ごとにtokenize

In [ ]:
nltk.sent_tokenize(sample_text)

* 単語ごとにtokenize
 * 問：この`nltk.word_tokenize`によるword tokenizationのメリットとデメリットは何か？

In [ ]:
print(nltk.word_tokenize(sample_text))

## spaCy

* spaCyも、Pythonの有名な自然言語処理ライブラリ。2015年スタートらしい。

* https://spacy.io/

### Tokenization

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
text_spacy = nlp(sample_text)

In [ ]:
[obj.text for obj in text_spacy.sents]

In [ ]:
text_spacy.sents

* 問： 下のword tokenizationは、先ほどの`nltk.word_tokenize`によるword tokenizationとどう違うか？

In [ ]:
print([obj.text for obj in text_spacy])

In [ ]:
tokens1 = list(nltk.word_tokenize(sample_text))
tokens2 = list(text_spacy)

In [ ]:
print(len(tokens1), len(tokens2))

## HTML文書

* __`<p>`__や__`<a>`__や__`<div>`__など、頻繁に使うHTMLタグは頭に入れておいてください。

* なぜなら、ある程度HTMLタグが読めてはじめて、スクレイピングのコードを書くための、HTMLソースの下調べができるからです。
 * 自前でWeb上から分析対象のテキストデータを取得するときは、ダウンロードしようとするWebページのHTMLの構造を自分の目で確認する。

* 問：誰かによって整備されたデータセットではなく、自前でHTML文書をスクレイピングすることのメリットとデメリットは何か？

### HTML文書のダウンロード
* いくつか方法はあるが、ここではrequestsモジュールを使う。

In [ ]:
import requests

data = requests.get('http://www.gutenberg.org/cache/epub/8001/pg8001.html')
content = data.text
print(content[2745:3948])

### Beautiful Soupの利用

* HTML文書の構造を解析するためによく使われるライブラリ。

* 参考資料：「Beautiful Soup 4によるスクレイピングの基礎」

 * https://www.atmarkit.co.jp/ait/articles/1910/18/news015.html

* 注意： 自動巡回やスクレイピングを禁止しているWebサイトもあるので注意しよう。

* HTMLタグを取り除くコードの例

In [ ]:
import re
from bs4 import BeautifulSoup

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    # 下の正規表現の意味を説明してみよう。
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

clean_content = strip_html_tags(content)
print(clean_content[1163:1957])

## 演習
* 上の`clean_content`を単語に分割し、各単語の出現頻度を求め、出現頻度の高い順に上位100の単語を、出現頻度とともに表示しよう。
* `clean_content`の内容を、すべて小文字に変換した後で、同じことをしてみよう。

In [ ]:
# 演習1の答案
text_spacy = nlp(clean_content)
tokens = [obj.text for obj in text_spacy]
print(tokens[10000:10020])
# 以下続けてコードを書く。

## 様々な前処理

### 特殊文字、数字、記号の除去

* reモジュールを使う。reはregular expression(正規表現)のこと。

* 問：テキストデータの前処理において特殊文字、数字、記号などを除去することのメリットとデメリットは何か？

* 問：下のセルで使われている２つの正規表現はそれぞれどういう意味か？

In [ ]:
import re

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


In [ ]:
s = "Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂"
s

In [ ]:
remove_special_characters(s, remove_digits=True)

In [ ]:
remove_special_characters(s)

### Stemming

* 語尾が変化する単語の、その変化を無くして、語幹を得る。
* 得られる語幹は、英単語として通用しない文字列になることが多い。
* Stemming and Lemmatization in Python
 * https://www.datacamp.com/community/tutorials/stemming-lemmatization-python


* 問：テキストデータの前処理としてstemmingをすることのメリットとデメリットは何か？
* 問：様々な種類のstemmerがあるのはなぜか？

* Porter Stemmerを使ってみる （stemmerと言えばこれ、というぐらい良く知られている。）

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

ps.stem('jumping'), ps.stem('jumps'), ps.stem('jumped')

In [ ]:
ps.stem('lying')

In [ ]:
ps.stem('strange')

### Lemmatization

* 動詞や形容詞は原型に、名詞は単数形に、等と、単語の元々の形に直すこと。

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [ ]:
s = 'The brown foxes are quick and they are jumping over the sleeping lazy dogs!'
s

In [ ]:
spacy_lemmatize_text(s)

## ストップワード

* ストップワードとは、非常に頻繁に使われるため、言語データの分析にあまり役に立ちそうにない単語のこと。
* これこそが英語のストップワードだ！と言えるような決定的なストップワードのリストがあるわけではない。
 * 主要なNLPライブラリでは、あらかじめ用意されたストップワードのリストを使うことができる。
 * しかし、分析したいテキストデータに合わせて、ストップワードのリストをカスタマイズすることも、よくある。

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

print(STOP_WORDS)
print(len(STOP_WORDS))

In [ ]:
def remove_stopwords(text, stopwords=None):
    if not stopwords:
        stopwords = spacy.lang.en.stop_words.STOP_WORDS
    tokens = [obj.text for obj in nlp(text)]
    filtered_tokens = [token for token in tokens if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
s = 'The brown foxes are quick and they are jumping over the sleeping lazy dogs!'
s

In [ ]:
remove_stopwords(spacy_lemmatize_text(s))

## 現代的なトークン化
* 図表は下記のブログ記事より。
 * https://ai.googleblog.com/2020/09/advancing-nlp-with-efficient-projection.html

![Segmentation.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/Segmentation.png)
![inherent_task_complexity.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/image3.png)

* 今は、tokenizationと言えば、ほぼ、サブワードに分けることを意味する。

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
tokenizer.tokenize('This framework generates embeddings for each input sentence')

# 課題2

* Wikipediaの適当な英語のエントリをダウンロードする。

 * 選ぶのが面倒という方はAIのエントリでもどうぞ。
   * https://en.wikipedia.org/wiki/Artificial_intelligence

* Beautiful Soupで本文のテキストだけを取得する。

 * HTMLのソースを見て、どこが本文かを確認する。
 * あるいは、ネット検索をして、Wikipediaのエントリから本文だけを取得する方法を調べる。

* 以下の前処理をする。

 * 大文字は小文字にする。

 * ストップワードを除去する。

 * lemmatizeする。

* 各単語の出現回数を求め、表示する。